# News Classification

In [44]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
#from TurkishStemmer import TurkishStemmer
from unicode_tr import unicode_tr

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc, f1_score, roc_auc_score

import warnings; warnings.simplefilter('ignore')

In [45]:
data = pd.read_csv("https://raw.githubusercontent.com/naynco/nayn.data/master/classification_clean.csv")
main_categories = ['DÜNYA', 'SPOR','SANAT','Teknoloji']
filter = data["Categories"].isin(main_categories)
data = data[filter]
train_data = data[['Title', 'Categories']]
train_data.head()

,Title,Categories
12006,58 Saniyede Katar Meselesi? Katar krizi nedir?...,DÜNYA
12496,58 Saniyede Türkiye - Almanya Gerginliği,DÜNYA
12877,"Adriana Lima, Bomba Aşkla İlgili İlk Kez Konuş...",DÜNYA
12878,Galatasaraylı Taraftarlar Patladı: İstifa Edin,SPOR
12880,"Galatasaray'dan Ayrılan Sabri, Neredeyse Bedav...",SPOR


In [46]:
def process_content(doc):
    #stemmer = TurkishStemmer()
    
    doc = unicode_tr(doc).lower()
    filter_punch = str.maketrans('', '', string.punctuation)
    stripped = doc.translate(filter_punch)

    clean_text = []
    for i in stripped.split():
        if i not in stopwords.words('turkish'):
            #clean_text.append(stemmer.stem(i))
            clean_text.append(i)

    return ' '.join(clean_text)

In [47]:
process_content("HarmonyOS 2.0 çalıştıran akıllı saat videosu sızdırıldı.")

'harmonyos 20 çalıştıran akıllı saat videosu sızdırıldı'

In [48]:
train_data['processed_title'] = train_data['Title'].apply(process_content)

In [49]:
train_data.head()

,Title,Categories,processed_title
12006,58 Saniyede Katar Meselesi? Katar krizi nedir?...,DÜNYA,58 saniyede katar meselesi katar krizi nedir v...
12496,58 Saniyede Türkiye - Almanya Gerginliği,DÜNYA,58 saniyede türkiye almanya gerginliği
12877,"Adriana Lima, Bomba Aşkla İlgili İlk Kez Konuş...",DÜNYA,adriana lima bomba aşkla ilgili ilk konuştu 35...
12878,Galatasaraylı Taraftarlar Patladı: İstifa Edin,SPOR,galatasaraylı taraftarlar patladı istifa edin
12880,"Galatasaray'dan Ayrılan Sabri, Neredeyse Bedav...",SPOR,galatasaraydan ayrılan sabri neredeyse bedavay...


In [50]:
categories = train_data['Categories']
titles = train_data['processed_title']
N = len(titles)
print('Number of news',N)

Number of news 11622


In [51]:
labels = list(set(categories))
n_classes = len(labels)
print('possible categories',labels)

possible categories ['SANAT', 'Teknoloji', 'DÜNYA', 'SPOR']


In [52]:
for l in labels:
    print('number of ',l,' news',len(train_data.loc[train_data['Categories'] == l]))

number of  SANAT  news 285
number of  Teknoloji  news 144
number of  DÜNYA  news 9226
number of  SPOR  news 1967


In [53]:
train_data['Categories'].value_counts()

DÜNYA        9226
SPOR         1967
SANAT         285
Teknoloji     144
Name: Categories, dtype: int64

In [54]:
X_train, X_test, y_train, y_test = train_test_split(train_data['processed_title'],train_data['Categories'],test_size=0.2,random_state=34)

In [55]:
#model = Pipeline([('vect', CountVectorizer()),
#                     ('tfidf', TfidfTransformer()),
#                     ('clf', LogisticRegression()),
#                     ])

In [56]:
#text_clf = model.fit(X_train, y_train)

In [57]:
#predicted = model.predict(X_test)

In [58]:
svc_tfidf = Pipeline([
        ("tfidf_vectorizer", TfidfVectorizer(stop_words=stopwords.words('turkish'), max_features=3000)),
        ("linear svc", SVC(kernel="linear"))
    ])

In [59]:
text_clf = svc_tfidf.fit(X_train, y_train)

In [60]:
predicted = svc_tfidf.predict(X_test)

In [61]:
confusion_matrix(y_test,predicted)

array([[1821,    2,   22,    0],
       [  54,    6,    0,    0],
       [  91,    0,  303,    0],
       [  26,    0,    0,    0]])

In [62]:
print('accuracy_score',accuracy_score(y_test,predicted))

accuracy_score 0.9161290322580645


In [63]:
print(classification_report(y_test, predicted, target_names=labels))

              precision    recall  f1-score   support

       SANAT       0.91      0.99      0.95      1845
   Teknoloji       0.75      0.10      0.18        60
       DÜNYA       0.93      0.77      0.84       394
        SPOR       0.00      0.00      0.00        26

    accuracy                           0.92      2325
   macro avg       0.65      0.46      0.49      2325
weighted avg       0.90      0.92      0.90      2325



In [64]:
cross_val_score(svc_tfidf, X_train, y_train, cv=5)

array([0.91612903, 0.91989247, 0.91823561, 0.91500807, 0.91662184])

In [65]:
cross_val_score(svc_tfidf, X_test, y_test, cv=5)

array([0.90322581, 0.88387097, 0.88172043, 0.88172043, 0.88172043])

In [69]:
def predict_title(model, new_data):
    test_data = pd.DataFrame(new_data, columns=['Title'])
    test_data['processed_title'] = test_data['Title'].apply(process_content)
    
    X_test = test_data['processed_title']
    predictions = model.predict(X_test)
    
    return predictions

In [70]:
t1 = ["Son Dakika | Galatasaray'da Arda Turan 9 yıl sonra ilk kez..."]
#t1 = ["27. İstanbul Caz Festivali başladı"]
news_title = pd.DataFrame(t1, columns=['Title'])
predict_title(svc_tfidf, t1)

array(['SPOR'], dtype=object)